# Calculate overshooting counts with the updated threshold
$T_b - T_{CP}$ !

In [1]:
import sys
sys.path.append("/home/b/b380887/cold-point-overshoot/python_scripts")

import xarray as xr
import get_d2_data as get_d2

from dask.diagnostics import ProgressBar


In [2]:
file_path = "/work/bb1153/b380887/global_tropics/"
out_path = file_path + "os_frequencies/"


#### Models

In [3]:
model_list = ["OBS"] #"SHIELD", "GEOS", "ICON", "SAM"] # , "SCREAM"]
chunks = {"time": 1}


In [4]:
diffs_dict = {}

for model in model_list:
    diffs_dict[model] = get_d2.get_diffs(model, file_path)


In [5]:
counts_dict = {}
freqs_dict = {}

for model in model_list:
    diffs = diffs_dict[model].chunk(chunks)
    counts_chunked = diffs.where(diffs < 0).count(dim="time")
    with ProgressBar():
        counts = counts_chunked.compute()
        
    ntime = len(diffs_dict[model].time)
    counts_dict[model] = counts
    freqs_dict[model] = counts/ntime
    
    print(model, "done")
    

[########################################] | 100% Completed | 15.9s
OBS done


In [9]:
def save_os_netcdf(model, counts_dict, freqs_dict, out_path, return_ds=False):
    """ 
    Make a dataset of os counts and frequencies for 
    one model and save as a netcdf
    """
    freq_da = xr.DataArray((freqs_dict[model]),
                           dims=counts_dict[model].dims,
                           coords = counts_dict[model].coords,
                           attrs={"long_name": "frequency of overshoot",
                                  "os_thresh": "Tb-Tcp < 0"
                                 }
                          )

    count_da = xr.DataArray(counts_dict[model],
                           dims=counts_dict[model].dims,
                           coords = counts_dict[model].coords,
                           attrs={"long_name": "number of occurrences of overshoot",
                                  "os_thresh": "Tb-Tcp < 0"
                                 }
                          )

    os_ds = xr.Dataset({"os_freq": freq_da, "os_count": count_da})
    
    out_name = out_path + "{m}_os_freq_count.nc".format(m=model)
    os_ds.to_netcdf(out_name)
    print("file saved to", out_name)
    
    if return_ds:
        return os_ds


In [7]:
for model in model_list:
    save_os_netcdf(model, counts_dict, freqs_dict, out_path)


file saved to /work/bb1153/b380887/global_tropics/os_frequencies/SHIELD_os_freq_count.nc
file saved to /work/bb1153/b380887/global_tropics/os_frequencies/GEOS_os_freq_count.nc
file saved to /work/bb1153/b380887/global_tropics/os_frequencies/ICON_os_freq_count.nc
file saved to /work/bb1153/b380887/global_tropics/os_frequencies/SAM_os_freq_count.nc


In [10]:
# just obs
for model in model_list:
    save_os_netcdf(model, counts_dict, freqs_dict, out_path)


file saved to /work/bb1153/b380887/global_tropics/os_frequencies/OBS_os_freq_count.nc
